In [32]:
# Import packages
import sys 
sys.path.append('./New functions & Dash GUI')
#import galaxy_functions
from dash import Dash, html, dash_table, dcc, callback, Output, Input
import dash
import numpy as np
import astropy.units as u
import pandas as pd
import plotly.express as px
from galaxy_functions import ElementInput, redshifted_wavelengthFunc, line_flux_at_z, sensScaled, Detected_Galaxies

# print(Detected_Galaxies)
# Incorporate data
line_table_data = np.load('/data/projects/FIRESIDE/data/line_table_cat_v9.npy', allow_pickle=True) #not sure if needed

# Initialize the app
app = Dash()

# App layout
app.layout = [
    html.Div(children='N of Galaxies Detected for Selected Element'),
    html.Hr(),
    html.Div(children=[
    dcc.RadioItems(options=['[CII] 158', '[OI] 145', '[NII] 122', '[OIII] 88', '[OI] 63', 
    '[NIII] 57', '[OIII] 52', '[SIII] 19', '[NeIII] 16', '[NeII] 13',
   '[OIV] 26', '[SIII] 33', '[SII] 35', '[SIV] 11', 'PAH 7.7',
   'PAH 11.3', 'Bralpha', 'Hmalpha', 'Pfalpa', 'Halpha',
   '[NeV] 14', '[NeV] 24'], value='[CII] 158', id='controls-and-radio-item')], style={"display":"inline-block", "gridTemplateColumns":"repeat(22, 1fr)", "gridGap":"10px","maxWidth":"100%", "margin":"auto", 'textAlign':"left"}), 
            
    html.Br(),
    html.Div(children=[
    html.Label('Min Wavelength'), dcc.Input(id='waveMin', type='number', value=25, placeholder='Enter min λ in microns'), 
    html.Label('Max Wavelength'), dcc.Input(id='waveMax', type='number', value=250, placeholder='Enter max λ in micron'), 
    html.Label('Inst. Sensitivity'), dcc.Input(id='sensitivity', type='number', value=10**-19, placeholder='Enter Instrument Sensitvity in Watts per sqaure meter'), 
    html.Label('Observation Time'), dcc.Input(id='tobsv', type='number', value=1000, placeholder='Enter Observation Time in Hours'), 
    html.Label('FOV'), dcc.Input(id='fov', type='number', value=1.00, placeholder='Enter FOV in sqaure degrees'),
    html.Label('Size Observed Area'), dcc.Input(id='area', type='number', value=1.96, placeholder='Enter Size of Observed Area in sqaure degrees'), 
    ], style={"display":"grid", "gridTemplateColumns":"repeat(6, 1fr)", "gridGap":"10px","maxWidth":"1000px", "margin":"auto"}), 
    dcc.Graph(figure={}, id='controls-and-graph')
]

# Add controls to build the interaction

@callback(
    Output(component_id='controls-and-graph', component_property='figure', allow_duplicate=True),
    Input(component_id='controls-and-radio-item', component_property='value'), 
    Input(component_id='waveMin', component_property='value'),
    Input(component_id='waveMax', component_property='value'), 
    Input(component_id='sensitivity', component_property='value'),
    Input(component_id='tobsv', component_property='value'),
    Input(component_id='fov', component_property='value'),
    Input(component_id='area', component_property='value'), prevent_initial_call='initial_duplicate',
)

def update_graph(Element, waveMin, waveMax, sensitivity, tobsv, fov, area):
    if waveMin is None or waveMax is None or sensitivity is None or tobsv is None or area is None or fov is None:
        raise dash.exceptions.PreventUpdate
    
    
    waveMin=waveMin*u.micron
    waveMax=waveMax*u.micron
    sensitivity=sensitivity*u.W/u.m**2
    tobsv=tobsv*u.hr
    area=area*u.degree**2
    fov=fov*u.degree**2
    

    line_index, catIndices, Total_Galaxies, FluxOfGalaxiesDetected = Detected_Galaxies(Element, line_table_data, waveMin, waveMax, sensitivity, tobsv, fov, area)
    #print(FluxOfGalaxiesDetected)
    
    
    #col_index=Assigned_Column_Element[selected_Element]
    #Galaxy_Detections_Element=line_table_data[:, col_index] not actual working function, placeholder
    
    x_val=FluxOfGalaxiesDetected
    fig = px.histogram(x=x_val, nbins=100, title=f"FluxOfGalaxiesDetected {Element}")
    return fig
#print(selected_Element)
# Run the app

if __name__ == '__main__':
    app.run(debug=True, port=8095)

In [2]:
Assigned_Column_Element = {
'[CII] 158':11,
'[OI] 145':line_table_data[:12], 
'[NII] 122':line_table_data[:13],
'[OIII] 88':line_table_data[:14],
'[OI] 63':line_table_data[:15] ,
'[NIII] 57':line_table_data[:16], 
'[OIII] 52':line_table_data[:17], 
'[SIII] 19':line_table_data[:18] ,
'[NeIII] 16':line_table_data[:18] ,
'[NeII] 13':line_table_data[:19],
'[OIV] 26':line_table_data[:20] ,
'[SIII] 33':line_table_data[:21],
'[SII] 35':line_table_data[:22],
'[SIV] 11':line_table_data[:23] ,
'PAH 7.7':line_table_data[:24],
'PAH 11.3':line_table_data[:25], 
'Bralpha':line_table_data[:26],
'Hmalpha':line_table_data[:27] ,
'Pfalpa':line_table_data[:28] ,
'Halpha':line_table_data[:29],
'[NeV] 14':line_table_data[:30], 
'[NeV] 24':line_table_data[:31] 
}

In [3]:
col_index=Assigned_Column_Element['[CII] 158']

In [4]:
x_valHist=line_table_data[:, col_index]

In [5]:
px.histogram(x=x_valHist, nbins=50)